In [5]:
import glob
import pandas as pd
import os
import json

import warnings
warnings.filterwarnings('ignore')

# Daily Data

In [6]:
# inputdir = "../../data/website/databricks/hourly"
# outputdir = "../../data/website/test/hourly"

# problematicpaths = []

# for tt in ["user", "spider"]:
#     for at in ["desktop", "mobile-dev", "mobile-app"]:
#         for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
#             for year in [2019]:
#                 for month in [9]:
#                     for day in range(1,32):

#                         jsondic = {i:[] for i in range(24)}
#                         filepathpattern = f"./{inputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}-*.csv"
#                         filepaths = glob.glob(filepathpattern)
                        
#                         if not filepaths:
#                             print(f"{filepathpattern} IS EMPTY")
#                             problematicpaths.append(filepathpattern)
#                             continue

#                         elif len(filepaths) != 24:
#                             print(f"{filepathpattern} DOES NOT CONTAIN 24 HOURS")
#                             problematicpaths.append(filepathpattern)
#                             continue
                        
#                         for filepath in filepaths:
#                             hour = int(filepath.split("-")[-1].split(".")[0])
#                             nested_csvs = glob.glob(filepath + "/*.csv")
#                             if len(nested_csvs) != 1:
#                                 print(f"{filepathpattern} CONTAINS NOT EXACTLY 1 NESTED CSV")
#                                 break
#                             realfilepath = nested_csvs[0]
#                             with open(realfilepath, "r") as f:
#                                 for line in f.read().split("\n"):
#                                     x, y = line.split(",")
#                                     jsondic[hour].append({"x":int(x), "y":int(y)})

#                         outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
#                         outfiledir = "/".join(outfilepath.split("/")[:-1])

#                         if not os.path.exists(outfiledir):
#                             os.makedirs(outfiledir)

#                         with open(outfilepath, "w") as f:
#                             json.dump(jsondic, f)

In [65]:
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/nice.parquet" "../../data/website/databricks/hourly2/sep2019.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/sep2018.parquet" "../../data/website/databricks/hourly2/sep2018.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/oct2019.parquet" "../../data/website/databricks/hourly2/oct2019.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/aug2019.parquet" "../../data/website/databricks/hourly2/aug2019.parquet"

In [66]:
df = pd.read_parquet("../../data/website/databricks/hourly2/aug2019.parquet").drop_duplicates()
df["date"] = pd.to_datetime(df["timestamp"], unit="s")
df["year"] = df.date.dt.year
df["month"] = df.date.dt.month
df["day"] = df.date.dt.day
df["hour"] = df.date.dt.hour
df

,timestamp,domain,trafficType,accessType,y,x,date,year,month,day,hour
0,1566896400,ru.wikipedia,spider,desktop,4,40,2019-08-27 09:00:00,2019,8,27,9
1,1566889200,ru.wikipedia,spider,mobile-web,1,3241,2019-08-27 07:00:00,2019,8,27,7
2,1566882000,en.wikipedia,spider,desktop,10,61,2019-08-27 05:00:00,2019,8,27,5
3,1566871200,en.wikipedia,spider,desktop,14,14,2019-08-27 02:00:00,2019,8,27,2
4,1566864000,en.wikipedia,user,desktop,3,330,2019-08-27 00:00:00,2019,8,27,0
...,...,...,...,...,...,...,...,...,...,...,...
463375,1567274400,es.wikipedia,user,mobile-web,37,1,2019-08-31 18:00:00,2019,8,31,18
463376,1567260000,es.wikipedia,user,mobile-web,5108,1,2019-08-31 14:00:00,2019,8,31,14
463377,1567252800,es.wikipedia,user,desktop,8,1,2019-08-31 12:00:00,2019,8,31,12
463378,1567252800,de.wikipedia,user,mobile-app,76,1,2019-08-31 12:00:00,2019,8,31,12


In [67]:
def df_to_json(df):
    outputdir = "../../data/website/hourly"

    jsondic = {i:[] for i in range(24)}

    # for hour in range(24):
    #     # cumx = 0
    #     for _, (x, y) in df[df["hour"]==hour][["x", "y"]].iterrows():
    #         # cumx += x
    #         jsondic[hour].append({"x":int(cumx), "y":int(y)})

    for _, (hour, x, y) in df[["hour", "x", "y"]].iterrows():
        jsondic[hour].append({"x":int(x), "y":int(y)})

    tt = df["trafficType"].iloc[0]
    at = df["accessType"].iloc[0]
    domain = df["domain"].iloc[0]
    year = df["year"].iloc[0]
    month = df["month"].iloc[0]
    day = df["day"].iloc[0]

    outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
    outfiledir = "/".join(outfilepath.split("/")[:-1])

    if not os.path.exists(outfiledir):
        os.makedirs(outfiledir)

    with open(outfilepath, "w") as f:
        json.dump(jsondic, f)

In [68]:
df\
.sort_values(["trafficType", "accessType", "domain", "year", "month", "day", "hour", "y"], ascending=[False, False, False, True, True, True, True, False])\
.groupby(["trafficType", "accessType", "domain", "year", "month", "day"])\
.apply(df_to_json)

""


In [191]:
# outputdir = "../../data/website/hourly"

# problematicpaths = []

# prefilter_df = df
# for tt in ["spider"]:#["user", "spider"]:
#     # prefilter_df = prefilter_df[(df["trafficType"] == tt)]
#     for at in ["mobile-web"]:#["desktop", "mobile-dev", "mobile-app"]:
#         # prefilter_df = prefilter_df[(df["accessType"] == at)]
#         for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
#             # prefilter_df = prefilter_df[(df["domain"] == domain)]
#             for year in [2019]:
#                 # prefilter_df = prefilter_df[(df["date"].dt.year == year)]
#                 for month in [9]:
#                     # prefilter_df = prefilter_df[(df["date"].dt.month == month)]
#                     for day in range(1,32):
#                         # prefilter_df = prefilter_df[(df["date"].dt.day == day)]

#                         jsondic = {i:[] for i in range(24)}

#                         # print(1)
#                         # for hour in range(24):

#                         #     # print(1)
#                         #     filtered_df = prefilter_df[
#                         #         (df["date"].dt.hour == hour)
#                         #     ]

#                         #     # filtered_df.sort_values("y", ascending=False, inplace=True)

#                         #     # print(2)
#                         #     cumx = 0
#                         #     for _, (x, y) in filtered_df[["x", "y"]].iterrows():
#                         #         cumx += x
#                         #         jsondic[hour].append({"x":int(cumx), "y":int(y)})

#                         # print(3)
#                         outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
#                         outfiledir = "/".join(outfilepath.split("/")[:-1])

#                         # print(4)
#                         if not os.path.exists(outfiledir):
#                             os.makedirs(outfiledir)

#                     #     print(5)
#                         with open(outfilepath, "w") as f:
#                             json.dump(jsondic, f)

#                     # 8/0

# Monthly Data

In [59]:
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/sep2018.parquet" "../../data/website/databricks/monthly/sep2018.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/sep2019.parquet" "../../data/website/databricks/monthly/sep2019.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/oct2019.parquet" "../../data/website/databricks/monthly/oct2019.parquet"
!dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/aug2019.parquet" "../../data/website/databricks/monthly/aug2019.parquet"

dbfs:/mnt/group09/websitedatamonthly/aug2019.parquet/_SUCCESS -> ../../data/website/databricks/monthly/aug2019.parquet/_SUCCESS
dbfs:/mnt/group09/websitedatamonthly/aug2019.parquet/_committed_4325917353023676521 -> ../../data/website/databricks/monthly/aug2019.parquet/_committed_4325917353023676521
dbfs:/mnt/group09/websitedatamonthly/aug2019.parquet/_started_4325917353023676521 -> ../../data/website/databricks/monthly/aug2019.parquet/_started_4325917353023676521
dbfs:/mnt/group09/websitedatamonthly/aug2019.parquet/part-00000-tid-4325917353023676521-cbb6381e-dd03-4e73-a66f-06c014253ba2-135330-1-c000.snappy.parquet -> ../../data/website/databricks/monthly/aug2019.parquet/part-00000-tid-4325917353023676521-cbb6381e-dd03-4e73-a66f-06c014253ba2-135330-1-c000.snappy.parquet
dbfs:/mnt/group09/websitedatamonthly/aug2019.parquet/part-00001-tid-4325917353023676521-cbb6381e-dd03-4e73-a66f-06c014253ba2-135331-1-c000.snappy.parquet -> ../../data/website/databricks/monthly/aug2019.parquet/part-0000

In [60]:
df = pd.read_parquet("../../data/website/databricks/monthly/aug2019.parquet").drop_duplicates()
df["date"] = pd.to_datetime(df["timestamp"], unit="s")
df

,timestamp,domain,trafficType,accessType,sumcount,date
0,1566741600,zh.wikipedia,user,desktop,5158,2019-08-25 14:00:00
1,1567249200,zh.wikipedia,spider,mobile-web,972,2019-08-31 11:00:00
2,1566849600,en.wikipedia,user,mobile-web,73373,2019-08-26 20:00:00
3,1566824400,fr.wikipedia,user,mobile-app,11734,2019-08-26 13:00:00
4,1566961200,es.wikipedia,spider,desktop,5205,2019-08-28 03:00:00
...,...,...,...,...,...,...
22315,1564833600,zh.wikipedia,user,desktop,16255,2019-08-03 12:00:00
22316,1566190800,es.wikipedia,user,mobile-web,8188,2019-08-19 05:00:00
22317,1566705600,ru.wikipedia,spider,mobile-web,2613,2019-08-25 04:00:00
22318,1565496000,zh.wikipedia,spider,desktop,6516,2019-08-11 04:00:00


In [61]:
df.sort_values("date")

,timestamp,domain,trafficType,accessType,sumcount,date
7953,1564617600,es.wikipedia,spider,desktop,4541,2019-08-01 00:00:00
9856,1564617600,fr.wikipedia,user,mobile-web,4887,2019-08-01 00:00:00
13872,1564617600,zh.wikipedia,user,mobile-web,1577,2019-08-01 00:00:00
17850,1564617600,ru.wikipedia,user,desktop,4310,2019-08-01 00:00:00
15018,1564617600,es.wikipedia,spider,mobile-web,1228,2019-08-01 00:00:00
...,...,...,...,...,...,...
8789,1567292400,fr.wikipedia,user,mobile-web,7789,2019-08-31 23:00:00
18175,1567292400,ru.wikipedia,user,mobile-web,2137,2019-08-31 23:00:00
10907,1567292400,en.wikipedia,user,mobile-app,122581,2019-08-31 23:00:00
6699,1567292400,de.wikipedia,user,mobile-web,3532,2019-08-31 23:00:00


In [62]:
outputdir = "../../data/website/monthly"

problematicpaths = []

for tt in ["user", "spider"]:
    for at in ["desktop", "mobile-web", "mobile-app"]:
        for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
            for year in [2019]:
                for month in [8]:

                    jsondic = []

                    filtered_df = df[
                        (df["domain"] == domain) &
                        (df["accessType"] == at) &
                        (df["trafficType"] == tt) &
                        (df["date"].dt.year == year) &
                        (df["date"].dt.month == month)
                    ]

                    filtered_df.sort_values("timestamp", ascending=True, inplace=True)

                    for _, (date, y) in filtered_df[["date", "sumcount"]].iterrows():
                        x = f"{date.year:04d}-{date.month:02d}-{date.day:02d}-{date.hour:02d}"
                        jsondic.append({"x":x, "y":int(y)})

                    outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}.json"
                    outfiledir = "/".join(outfilepath.split("/")[:-1])  # the directory where the file is in

                    if not os.path.exists(outfiledir):
                        os.makedirs(outfiledir)

                    with open(outfilepath, "w") as f:
                        json.dump(jsondic, f)

In [76]:
# !rm ../../data/website/hourly/*/*/*/2019-10*.json
# !rm ../../data/website/monthly/*/*/*/2019-10.json

The visualisation we prepared shows 2 plots. The top plot is the hourly page views over 1 month. The y-axis are the amount of views in a given hour, and the x-axis represents all hours of the month.

On the plot below is the page view distribution at a specific hour. The x-axis represents the pages and the y-axis shows the page views. The pages are sorted according to their number of views, and their titles are left out as the focus is on the distribution. 

The plot is log-log to highlight the power law distribution - few pages have many views (top-left corner) while many pages only have few views (bottom-right corner).

We can pick certain days and compare their traffic over time with the slider

-- slide around, any 2 days --

We also can select multiple filters, for example which traffic we want to see or which access type.

Now to go back to our research question, we want to see whether there are any artefacts of the attack on September 6th 2019. 

Lets pick this date and some date prior and after that.

-- pick 2019-09-06, 2019-09-03, 2019-09-10 --

We also know the attack was most pronounced on european servers, so lets assume that these european servers mostly contain european domains. Lets pick the german domain.

-- picke de.wikipedia --

Lets also pick all traffic types and all access types just to be sure we are not missing anything

-- pick all traffic type, all access type --

Now the cool thing about our visualisation is that we can get a quick overview of all the page views on these days.

-- move hour slider --

We see that there is a big drop in traffic in the evening. its also visible in the top plot.

However more relevant and unfortunate is that there is no weird traffic before or during the beginning of the attack. No pages were viewed significantly more often, the distributions are identical to other days. 

